# A Guided Tour of LSST Calexps
<br>Owner(s): **David Shupe** ([@stargaser](https://github.com/LSSTScienceCollaborations/StackClub/issues/new?body=@stargaser))
<br>Last Verified to Run: **2018-08-07**
<br>Verified Stack Release: **v16.0** (also lsst_w_2018_31, with `getName` modification)

We'll inspect a visit image ``calexp`` object, and then show how a coadd image differs.

### Learning Objectives:

After working through this tutorial you should be able to follow some best practices when working with LSST ``calexp`` (image) objects.

### Logistics
This notebook is intended to be runnable on `lsst-lspdev.ncsa.illinois.edu` from a local git clone of https://github.com/LSSTScienceCollaborations/StackClub.

## Set-up

In [ ]:
from lsst.daf.persistence import Butler

In [ ]:
import lsst.afw.display as afw_display

### Retrieving and inspecting a calexp

For the first part of this tutorial we will use simulated LSST data from Twinkles, see https://github.com/LSSTDESC/Twinkles/blob/master/README.md

Define a data directory and create a Butler

In [ ]:
datadir = '/home/YOUR_USER_NAME/ci_hsc_small/rerun/YOUR_RERUN_NAME'
butler = Butler(datadir)

Other notebooks show how to view what data are available in a Butler object. Here we get a specific one.

In [ ]:
dataId = {'visit': 903334, 'ccd': 16}
calexp = butler.get('calexp', **dataId)

In terms of pixel data, a calexp contains an image, a mask, and a variance.

Let's see how to access the image.

In [ ]:
calexp.image

To show the pixel data, we will make use of the matplotlib backend to `lsst.afw.display`.

Due to current limitations of this backend, the display must be defined and used in the same code cell, much as matplotlib commands in a notebook must all be in one cell to produce a plot.

In [ ]:
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.dpi'] = 120

If the entire calexp is displayed, masks will be overlaid. Here we will eschew the mask display by showing only the image.

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale("asinh", "zscale")
display1.mtv(calexp.image)

To access the pixel values as an array, use the `.array` attribute.

In [ ]:
data = calexp.image.array
data

In [ ]:
data.__class__

Let's list all the methods for our calexp.

In [ ]:
calexp_methods = [m for m in dir(calexp) if not m.startswith('_')]

In [ ]:
calexp_methods

Access the masked Image

In [ ]:
calexp.maskedImage

Access the variance object and the underlying Numpy array

In [ ]:
calexp.variance

In [ ]:
calexp.variance.array

Access the mask and its underlying array

In [ ]:
calexp.mask

In [ ]:
calexp.mask.array

Get the dimensions of the image, mask and variance

In [ ]:
calexp.getDimensions()

The image, maskedImage and Exposure objects in `lsst.afw.display` include information on **LSST pixels**, which are 0-based with an optional offset.

For a calexp these are usually zero.

In [ ]:
calexp.getXY0()

In [ ]:
calexp.getX0(), calexp.getY0()

Access the wcs object

In [ ]:
wcs = calexp.getWcs()
wcs

The WCS object can be used e.g. to convert pixel coordinates into sky coordinates

In [ ]:
wcs.pixelToSky(100.0, 100.0)

Let's try accessing the metadata, and see what (header) keywords we have.

Check if our calexp has a PSF

In [ ]:
calexp.hasPsf()

In [ ]:
psf = calexp.getPsf()

The PSF object can be used to get a realization of a PSF at a specific point

In [ ]:
from lsst.afw.geom import Point2D
psfimage = psf.computeImage(Point2D(100.,100.))

Access the calibration object which can be used to convert instrumental magnitudes to AB magnitudes

In [ ]:
calib = calexp.getCalib()
calib

### Image cutouts

We can make a cutout from the calexp in our session.

In [1]:
import lsst.afw.geom as afwGeom
import lsst.afw.image as afwImage

In [ ]:
bbox = afwGeom.Box2I()
bbox.include(afwGeom.Point2I(200,200))
bbox.include(afwGeom.Point2I(800,800))
cutout = calexp[bbox]

Notice that when the image is displayed, the pixel values relate to the parent image.

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale("asinh", "zscale")
display1.mtv(cutout.image)

The coordinate of the lower-left-hand pixel is XY0.

In [ ]:
cutout.getXY0()

If a cutout was all that was desired from the start, we could have used our BoundingBox together with our Butler to have read in only the cutout.

The `clone` method makes a deep copy. The result can be sliced with a BoundingBox

In [ ]:
clone_cutout = calexp.clone()[bbox]

In [ ]:
display1 = afw_display.Display(frame=1, backend='matplotlib')
display1.scale("asinh", "zscale")
display1.mtv(clone_cutout.image)